## 导入依赖库

In [86]:
# 导入Python标准库中的json模块，用于处理JSON数据
import json
# 导入Python标准库中的os模块，用于操作系统功能，如环境变量
import os

# 从qwen_agent.llm模块导入get_chat_model函数
from qwen_agent.llm import get_chat_model

## 模拟一个外部工具函数

In [87]:
# 示例虚拟函数，硬编码为返回相同的天气
# 在生产环境中，这可以是您的后端API或外部API
def get_current_weather(location, unit='华氏度'):
    """获取给定地点的当前天气"""
    if '东京' in location.lower():
        return json.dumps({'location': '东京', 'temperature': '10', 'unit': '摄氏度'})
    elif '旧金山' in location.lower():
        return json.dumps({'location': '旧金山', 'temperature': '72', 'unit': '华氏度'})
    elif '巴黎' in location.lower():
        return json.dumps({'location': '巴黎', 'temperature': '22', 'unit': '摄氏度'})
    else:
        return json.dumps({'location': location, 'temperature': '未知'})

## 测试Qwen2-7B-Instruct 的OpenAI风格的API
备注：先采用vLLM进行Qwen2-7B模型部署，启用OpenAI风格的API接口，具体参考：《部署高效AI模型：使用vLLM进行Qwen2-7B模型推理》

In [88]:
from openai import OpenAI
openai_api_key = "sk-xxx" # 随便填写，只是为了通过接口参数校验
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_outputs = client.chat.completions.create(
    model="Qwen2-7B-Instruct",
    messages=[
        {"role": "system", "content": "你是一个智能AI助手"},
        {"role": "user", "content": "你好"},
    ]
)
print(chat_outputs)

ChatCompletion(id='cmpl-2e9883ed58154fa5bc6ff63886aea585', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='你好！很高兴能帮助你。有什么问题或需要我提供的信息吗？', role='assistant', function_call=None, tool_calls=None), stop_reason=None)], created=1719843868, model='Qwen2-7B-Instruct', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=19, total_tokens=36))


## 定义LLM

In [89]:
llm = get_chat_model({
     'model': 'Qwen2-7B-Instruct',
     'model_server': 'http://localhost:8000/v1',  # api_base
     'api_key': 'EMPTY',
    # 使用 DashScope 提供的模型服务：
    #'model': 'qwen1.5-14b-chat',
    #'model_server': 'dashscope',
    #'api_key': os.getenv('DASHSCOPE_API_KEY'),

    # 使用 DashScope 提供的与 OpenAI 兼容的模型服务：
    # 'model': 'qwen1.5-14b-chat',
    # 'model_server': 'https://dashscope.aliyuncs.com/compatible-mode/v1', 
    # 'api_key': os.getenv('DASHSCOPE_API_KEY'),

    # 使用 Together.AI 提供的模型服务：
    # 'model': 'Qwen/Qwen1.5-14B-Chat',
    # 'model_server': 'https://api.together.xyz',   # api_base
    # 'api_key': os.getenv('TOGETHER_API_KEY'),
})

## 定义工具函数信息

In [90]:
functions = [{
        'name': 'get_current_weather',
        'description': '获取给定位置的当前天气',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': '城市和州，例如加利福尼亚州旧金东京山',
                },
                'unit': {
                    'type': 'string',
                    'enum': ['摄氏度', '华氏度']
                },
            },
            'required': ['location'],
        },
    }]

## 定义messages消息

In [91]:
messages = [{'role': 'user', 'content': "东京的天气怎么样？"}]

## 第一次请求大模型（查找工具函数）

In [92]:
print('# 助手回复 1：')
responses = []
for responses  in llm.chat(
        messages=messages,
        functions=functions,
        stream=True,#True
        # 注意：extra_generate_cfg 是可选的
        # extra_generate_cfg=dict(
        #     # 注意：如果 function_choice='auto'，让模型决定是否调用函数
        #     # function_choice='auto',  # 如果没有设置 function_choice，默认是 'auto'
        #     # 注意：设置 function_choice='get_current_weather' 强制模型调用此函数
        #     function_choice='get_current_weather',
        # ),
):
    print(responses)

# 助手回复 1：
[{'role': 'assistant', 'content': '', 'function_call': {'name': 'get', 'arguments': ''}}]
[{'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current', 'arguments': ''}}]
[{'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current_weather', 'arguments': ''}}]
[{'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current_weather', 'arguments': ''}}]
[{'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current_weather', 'arguments': ''}}]
[{'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current_weather', 'arguments': ''}}]
[{'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current_weather', 'arguments': ''}}]
[{'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current_weather', 'arguments': ''}}]
[{'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current_weather', 'arguments': '{"'}}]
[{'role': 'assistant', 'content': '', 'function_call': {'n

## 将响应内容放入消息中备用

In [93]:
messages.extend(responses)  # 用助手的回复扩展对话

In [94]:
messages

[{'role': 'user', 'content': '东京的天气怎么样？'},
 {'role': 'assistant',
  'content': '',
  'function_call': {'name': 'get_current_weather',
   'arguments': '{"location": "东京", "unit": "摄氏度"}'}}]

## 第二次大模型调用
1、取出工具函数进行手工调用 2、将响应结果给到大模型进行整理增强

In [95]:
# 第 2 步：检查模型是否想要调用一个函数
last_response = messages[-1]
# 检查变量 last_response 是否包含键 function_call 并且其值不为 None
if last_response.get('function_call', None):

    # 第 3 步：调用函数
    # 注意：JSON 响应可能不是有效的；确保要处理错误
    available_functions = {
        'get_current_weather': get_current_weather,
    }  # 此示例中只有一个函数，但您可以有多个
    function_name = last_response['function_call']['name']
    function_to_call = available_functions[function_name]
    function_args = json.loads(last_response['function_call']['arguments'])
    function_response = function_to_call(
        location=function_args.get('location'),
        unit=function_args.get('unit'),
    )
    print('# 函数响应:')
    print(function_response)

    # 第 4 步：将每个函数调用和函数响应的信息发送给模型
    messages.append({
        'role': 'function',
        'name': function_name,
        'content': function_response,
    })  # 用函数响应扩展对话

    print('# 助手回复 2:')
    for response in llm.chat(
            messages=messages,
            functions=functions,
            stream=True,#True
    ):  # 在模型能够看到函数响应的情况下获取模型的新回复
        print(response)

# 函数响应:
{"location": "\u4e1c\u4eac", "temperature": "10", "unit": "\u6444\u6c0f\u5ea6"}
# 助手回复 2:
[{'role': 'assistant', 'content': '东京'}]
[{'role': 'assistant', 'content': '东京的'}]
[{'role': 'assistant', 'content': '东京的当前'}]
[{'role': 'assistant', 'content': '东京的当前气温'}]
[{'role': 'assistant', 'content': '东京的当前气温是'}]
[{'role': 'assistant', 'content': '东京的当前气温是1'}]
[{'role': 'assistant', 'content': '东京的当前气温是10'}]
[{'role': 'assistant', 'content': '东京的当前气温是10摄'}]
[{'role': 'assistant', 'content': '东京的当前气温是10摄氏'}]
[{'role': 'assistant', 'content': '东京的当前气温是10摄氏度'}]
[{'role': 'assistant', 'content': '东京的当前气温是10摄氏度。'}]


In [96]:
response[-1]

{'role': 'assistant', 'content': '东京的当前气温是10摄氏度。'}